In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My\ Drive/Colab_notebooks/Official_GitHub_Repo_LV

/content/gdrive/My Drive/Colab_notebooks/Official_GitHub_Repo_LV


In [3]:
# !unzip pke.zip

In [4]:
# !git clone https://github.com/boudinfl/ake-datasets.git

In [5]:
# !git clone http://github.com/stanfordnlp/glove

In [6]:
# %cd glove

In [7]:
# !make

In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import SnowballStemmer

import pke
from pke import LoadFile

import os
from os import listdir

import string

import sys

import numpy as np

import operator

import re

from Functions import Functions


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
mode = 'test'
# 'train'
# 'test'

pca_dim = 'NoPCA'

In [10]:
stopwords_extend = []
with open('./stopwords_snowball_expanded.txt') as f:
    stopwords_extend = f.read().splitlines()
stopwords_list = stopwords_extend
stopwords_list += list(string.punctuation)
stopwords_list += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stopwords_list += stopwords.words('english')
stopwords_list_stemmed = []
ps = SnowballStemmer('porter')
for stopw in stopwords_list:
    stopwords_list_stemmed.append(ps.stem(stopw))
stopwords_list += stopwords_list_stemmed
stopwords_list.extend(['\t', '\n', '\x0b', '\x0c', '\r', ' '])
common_adjectives = []
with open('./common_adjectives.txt') as f:
    common_adjectives = f.read().splitlines()
print(common_adjectives)
reporting_verbs = []
with open('./reporting_verbs.txt') as f:
    reporting_verbs = f.read().splitlines()
print(reporting_verbs)
determiners = []
with open('./determiners.txt') as f:
    determiners = f.read().splitlines()
print(determiners)
functional_words = []
with open('./functional_words.txt') as f:
    functional_words = f.read().splitlines()
print(functional_words)

stopwords_with_capital = [stop.title() for stop in stopwords_list]
stopwords_list.extend(stopwords_with_capital)
stopwords_list = list(set(stopwords_list))

['other', 'new', 'good', 'great', 'big', 'small', 'large', 'different', 'little', 'important', 'bad', 'real', 'best', 'only', 'able', 'major', 'better', 'possible', 'true', 'easy', 'clear', 'recent', 'certain', 'difficult', 'available', 'likely', 'current', 'wrong', 'past', 'fine', 'common', 'poor', 'significant', 'similar', 'same', 'happy', 'serious', 'ready', 'various', 'entire', 'close', 'final', 'main', 'nice', 'huge', 'popular']
['admit', 'agree', 'believe', 'claim', 'confess', 'decide ', 'deny', 'doubt', 'explain', 'feel', 'hope', 'insist', 'mention', 'promise', 'reply', 'say', 'suggest', 'advise', 'prefer', 'propose', 'urge', 'accuse', 'apologise', 'prohibit', 'refuse', 'threaten', 'ask', 'allow', 'beg', 'encourage', 'forbid', 'instruct', 'invite', 'persuade', 'remind', 'tell', 'warn', 'want']
['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 'his', 'her', 'its', 'our', 'their', 'few', 'little', 'much', 'many', 'lot', 'most', 'some', 'any', 'enough', 'all', 'bot

In [14]:
dataset_path = '/content/gdrive/My Drive/Colab_notebooks/LocalRepresentationsKE/ake-datasets/datasets/NUS/'

dataset = dataset_path.split('/')[len(dataset_path.split('/'))-2]

In [15]:
keyphrases_folder = 'references/'

files_subfolder = ''
data_files = []
if mode == 'train':
  if dataset == 'NUS':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:111]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_combined_stem
  elif dataset == 'SemEval-2010':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_author_stem = 'train.author.stem.json'
    train_reader_stem = 'train.reader.stem.json'
    train_combined_stem = 'train.combined.stem.json'

    default_keyphrases_file = train_combined_stem
  elif dataset == 'ACM':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[400:2304]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'PubMed':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[320:]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'Citeulike-180':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[80:]
    test_reader_stem = 'test.reader.stem.json'
    
    default_keyphrases_file = test_reader_stem
  elif dataset == 'CSTR':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_author_stem = 'train.author.stem.json'
    
    default_keyphrases_file = train_author_stem

elif mode == 'test':
  dataset = dataset_path.split('/')[len(dataset_path.split('/'))-2]
  if dataset == 'NUS':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[111:211]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'SemEval-2010':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'ACM':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:400]
    test_author_stem = 'test.author.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'PubMed':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:320]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'Citeulike-180':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:80]
    test_reader_stem = 'test.reader.stem.json'
    
    default_keyphrases_file = test_reader_stem
  elif dataset == 'CSTR':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem


print('Dataset:', dataset, 'mode:', mode, 'Number of files:', len(data_files), 'Sorted data file names:', data_files)    

# model = fasttext.load_model("/content/gdrive/My Drive/Colab_notebooks/LocalRepresentationsKE/Fasttext/model_original_final.bin")

Dataset: NUS mode: test Number of files: 100 Sorted data file names: ['201.xml', '202.xml', '203.xml', '204.xml', '205.xml', '206.xml', '207.xml', '208.xml', '209.xml', '21.xml', '210.xml', '211.xml', '212.xml', '213.xml', '214.xml', '215.xml', '22.xml', '23.xml', '24.xml', '25.xml', '26.xml', '27.xml', '28.xml', '29.xml', '3.xml', '30.xml', '31.xml', '32.xml', '33.xml', '34.xml', '35.xml', '36.xml', '37.xml', '38.xml', '39.xml', '4.xml', '40.xml', '41.xml', '42.xml', '43.xml', '44.xml', '45.xml', '46.xml', '47.xml', '48.xml', '49.xml', '5.xml', '50.xml', '51.xml', '52.xml', '53.xml', '54.xml', '55.xml', '56.xml', '57.xml', '58.xml', '59.xml', '6.xml', '60.xml', '61.xml', '62.xml', '63.xml', '64.xml', '65.xml', '66.xml', '67.xml', '68.xml', '69.xml', '7.xml', '70.xml', '71.xml', '72.xml', '73.xml', '74.xml', '75.xml', '76.xml', '77.xml', '78.xml', '79.xml', '8.xml', '80.xml', '81.xml', '82.xml', '83.xml', '84.xml', '85.xml', '86.xml', '87.xml', '88.xml', '89.xml', '9.xml', '90.xml', '9

In [16]:
percentage_LV_list = {}

precision_LV_list = {}

fm_LV_list = {}


for top in range(5, 21, 5):

    percentage_LV_list[top] = []

    precision_LV_list[top] = []

    fm_LV_list[top] = []

for fvc, filevc in enumerate(data_files):
  functions_obj = Functions(dataset_path+files_subfolder, filevc)
  print(fvc, filevc)
  #filevc = '208.xml'

  keyphrases = functions_obj.document_gold_keyphrases(dataset_path+keyphrases_folder+default_keyphrases_file)
  print(keyphrases)      
  if len(keyphrases) == 0:# or filevc == '29080.xml':
    print('No keyphrase for this file...')
    continue  
  flatten_keywords = [word for sublist in keyphrases for word in sublist]
  flatten_keywords = list(set(flatten_keywords))
  print(len(flatten_keywords), flatten_keywords)

  extractor = LoadFile(dataset_path+files_subfolder+filevc)
  doc_text = extractor.read_document_from_xml_file()
  # print(text_doc)
  doc_text = doc_text.replace('\n', ' ')
  doc_text = re.sub("\s\s+", " ", doc_text)#.translate(str.maketrans('', '', string.punctuation)))
  # load the content of the document, here in txt format
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  sentences = extractor.load_document(input=doc_text,
                                      language="en",
                                      normalization='stemming')

  clean_sentences, clean_bi_sentences, clean_tri_sentences, dict_realSentId_matrixSentId = extractor.get_clean_sentence(
      stopwords_list, common_adjectives, reporting_verbs, determiners, functional_words)
  
  # for id, sent in clean_sentences.items():
  #   print(id, sent)
  #   if id == 10:
  #     break

  # extractor.ngram_selection(n=3)
  
  position_matrix, vocab, ivocab = extractor.ngram_position_matrix()
  print("position_matrix preview", position_matrix, position_matrix.shape)
  # print(position_matrix.shape, len(vocab.keys()), len(clean_sentences.keys()))

  cooccurrence_dict = extractor.build_cooccurrences(window=10)
  cooccurrence_matrix = extractor.get_cooccurrences_matrix()
  # print("cooccurrence_matrix preview", cooccurrence_matrix, cooccurrence_matrix.shape)
  # print(cooccurrence_matrix.shape, len(vocab.keys()), len(vocab.keys()))
  pca_cooccurrence_matrix = np.zeros(shape=(1, 1))
  if pca_dim == 'NoPCA':
    pca_cooccurrence_matrix = cooccurrence_matrix
  else:
    pca_cooccurrence_matrix = extractor.pca_projection(cooccurrence_matrix, pca_dim)

  
  # LV_topN: topN LV - original score,

  LV_topN = extractor.tune_SC_cooccurrences(
      position_matrix, pca_cooccurrence_matrix, ivocab, score_type = 'custom', measure = 'euclidean')
  print('all LV are:', len(LV_topN), 'out of', len(ivocab.keys()))
  #LV_topN_list = list(dict(LV_topN).keys())
  print('OUTPUT OF LV:', LV_topN)
  LV_topN_list_all = []
  for u in LV_topN:
      LV_topN_list_all.append(u[0])


  for top in range(5, 21, 5):
    print('---------------------------------------------------------------------------TOP:', top, '----------------------------------------------------------------------------------------------------------------------------------------------------------')

    LV_topN_list = LV_topN_list_all[:top]

    percentage_LV = len(set(LV_topN_list).intersection(flatten_keywords))/float(len(flatten_keywords))

    precision_LV = 0.0
    if len(LV_topN_list) > 0: 
      precision_LV = len(set(LV_topN_list).intersection(flatten_keywords))/float(len(LV_topN_list))

    fm_LV = 0.0
    if percentage_LV > 0 or precision_LV > 0:
      fm_LV = 2.0*(percentage_LV*precision_LV)/(percentage_LV+precision_LV)


    print(top, ': rec_LV', percentage_LV)

    print(top, ': prec_LV', precision_LV)

    print(top, ': fm_LV', fm_LV)

    print('---------------------------------------------------------------------------AVERAGE----------------------------------------------------------------------------------------------------------------------------------------------------------')

    percentage_LV_list[top].append(percentage_LV)

    precision_LV_list[top].append(precision_LV)

    fm_LV_list[top].append(fm_LV)


    print(top, ': rec_LV_av:', sum(percentage_LV_list[top])/len(percentage_LV_list[top])) 

    print(top, ': prec_LV_av:', sum(precision_LV_list[top])/len(precision_LV_list[top]))

    print(top, ': fm_LV_av:', sum(fm_LV_list[top])/len(fm_LV_list[top]))

    print('')
                                  

Output hidden; open in https://colab.research.google.com to view.